In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import time
import winsound

In [2]:
start = time.time()

# 尋找上市櫃公司完整名稱
https://mops.twse.com.tw/mops/web/t51sb01
如果檔案存在的話，就直接讀取檔案，不用再爬取一次
檔案必須用selenium爬取，因為網頁是用javascript產生的，檔名隨時會更改

In [3]:
# 年薪中位數100萬以上，不論員工人數
high_company_name_list = [
    '聯詠',	'鴻準',	'聯發科',	'原相',	'揚智',	'智擎',	'新潤',	'群聯',	'瑞昱',	'華固',	'創意',	'和泰車',	'力旺',	'聯陽',	'M31',	'矽創',	'台積電',	'義隆',	'晶豪科',	'致新',	'凌通',	'鴻海',	'是方',	'杰力',	'群光',	'九暘',	'宏觀',	'中華電',	'環球晶',	'中鋼',	'祥碩',	'廣宇',	'致伸',	'神基',	'智原',	'聚積',	'菱光',	'智易',	'創惟',	'敦泰',	'新興',	'亞泰',	'中聯資源',	'松翰',	'台苯',	'愛普',	'晶心科',	'緯穎',	'威盛',	'台塑化',	'神盾',	'華票',	'台塑',	'南帝',	'大統益',	'唐榮公司',	'順達',	'信驊',	'台化',	'凌陽',	'華晶科',	'合庫金',	'漢唐',	'新海',	'華航',	'南亞',	'裕日車',	'通嘉',	'亞聚',	'佳世達',	'弘塑',	'達興材料',	'中碳',	'華邦電',	'勤誠',	'精成科',	'群電',	'晶相光',	'彰銀',	'天鈺',	'華碩',	'大中',	'鉅邁',	'南亞科',	'基泰',	'中信金',	'台達電',	'東生華',	'至上',	'廣達',	'上海商銀',	'嘉威',	'國喬',	'金麗科',	'迅杰',	'裕民',	'宏普',	'兆豐金',	'第一金',	'嘉泥',	'高雄銀',	'宏碁',	'應廣',	'茂綸',	'同亨',	'盛群',	'展匯科',	'台肥',	'矽統',	'緯創',	'崇越電',	'偉詮電',	'九齊',	'亞泥',	'台船',	'微星',	'旭隼',	'蔚華科',	'聚陽',	'新唐',	'立積',	'中再保',	'聯傑',	'中航',	'萬海',	'聖暉',	'仁寶',	'崇越',	'茂達',	'華擎',	'光寶科',	'昇陽',	'鈊象',	'華南金',	'逸達',	'華立',	'驊訊',	'中鼎',	'國泰金',	'台翰',	'富邦金',	'遠東銀',	'松上',	'擎亞',	'大聯大',	'台汽電',	'台開',	'順藥',	'臺企銀',	'凡甲',	'安國',	'晶宏',	'台勝科',	'長榮航',	'台聚',	'億豐',	'統一',	'樺漢',	'亞信',	'龍邦',	'益登',	'志超',	'藍天',	'三圓',	'中光電',	'通泰',	'王道銀行',	'嘉澤',	'文曄',	'廣明',
]

In [4]:
# 年薪中位數85萬~100萬，員工人數破百
middle_company_name_list = [
    '類比科',	'神達',	'台達化',	'光菱',	'佳能',	'新普',	'和碩',	'旺宏',	'資通',	'海悅',	'欣泰',	'達方',	'東洋',	'鈺創',	'台新金',	'元大期貨',	'聯電',	'圓剛',	'永豐金',	'中華',	'一零四',	'訊連',	'國建',	'禾瑞亞',	'玉山金',	'裕融',	'達欣工',	'威剛',	'帆宣',	'仲琦',	'中鋼構',	'大台北',	'均豪',	'友訊',	'萬潤',	'聯成',	'鑫創',	'長榮',	'友華',	'碩天',	'裕隆',	'關貿',	'台中銀',	'豐泰',	'台微體',	'智崴',	'永大',	'瑞穎',	'中菲',	'瑞軒',	'安勤',	'華夏',	'元太',	'東友',	'大塚',	'英業達',	'晉泰',	'開發金',	'世界',	'台航',	'正崴',	'陽明',	'牧德',	'聰泰',	'台灣大',	'盟立',	'金寶',	'京鼎',	'由田',	'鼎天',	'台橡',	'東森',	'漢翔',	'威健',	'雙鴻',	'元大金',	'穩懋',	'東聯',	'德律',	'系微',	'新鼎',	'建漢',	'盛餘',	'和益',	'敦陽科',	'巨路',	'精測',	'喬鼎',	'海華',	'凌華',	'朋億',	'旭富',	'台泥',	'宏正',	'安馳',	'新產',	'承業醫',	'政伸',	'居易',	'偉訓',	'旺矽',	'大江',	'豪勉',	'閎康',	'藥華藥',	'研華',	'致茂',	'豐興',	'崑鼎',	'東和鋼鐵',	'信錦',
]

In [5]:
# 建築營造年薪中位數80萬以上，不論員工人數
construction_company_name_list = [
    '新潤',	'華固',	'基泰',	'宏普',	'昇陽',	'台開',	'三圓',	'鑫龍騰',	'華建',	'國建',	'順天',	'達欣工',	'愛山林',	'永信建',	'理銘',	'冠德',	'士開',	'京城',	'國揚',	'力麒',	'遠雄',	'隆大',	'日勝生',	'建國',	'三豐',	'宏璟',
]

In [6]:
search_company = []
for i in high_company_name_list:
    search_company.append('https://www.104.com.tw/jobs/search/?keyword={}'.format(i))

In [16]:
company_dict = {}

for index, item in enumerate(search_company):
    res = requests.get(item)
    time.sleep(0.5)
    soup = bs(res.text, 'lxml')
    try:
        company_name = soup.select('div > a.d-none.d-md-inline.jb-link.jb-link-blue.jb-link-blue--visited.h2.mb-1')[0].text
        company_main_page = soup.select('div > a.d-none.d-md-inline.jb-link.jb-link-blue.jb-link-blue--visited.h2.mb-1')[0]['href']
        company_dict.update({company_name : company_main_page})
        print('第' + str(index) + '爬完' + '\t\t' + company_name)
    except:
        pass
        print('第' + str(index) + '找不到' + '\t' + high_company_name_list[index])
print('共' + str(len(company_dict)) + '家公司')

第0爬完		聯詠科技股份有限公司
第1爬完		鴻準精密工業股份有限公司
第2爬完		聯發科技股份有限公司
第3爬完		原相科技股份有限公司
第4爬完		揚智科技股份有限公司
第5爬完		FUNDAY_智擎數位科技股份有限公司
第6爬完		新潤興業股份有限公司
第7爬完		Phison Electronics Corp_群聯電子股份有限公司
第8爬完		瑞昱半導體股份有限公司
第9爬完		華固建設股份有限公司
第10找不到	創意
第11爬完		和泰車體製造股份有限公司
第12爬完		力旺電子股份有限公司
第13爬完		聯陽半導體股份有限公司
第14爬完		M31 Technology Corporation_円星科技股份有限公司
第15爬完		矽創電子股份有限公司
第16爬完		台灣積體電路製造股份有限公司(台積電)
第17爬完		義隆電子股份有限公司
第18爬完		晶豪科技股份有限公司
第19爬完		致新科技股份有限公司
第20爬完		凌通科技股份有限公司
第21爬完		鴻海保全股份有限公司
第22爬完		是方電訊股份有限公司(中華電信關係企業)
第23爬完		杰力科技股份有限公司
第24爬完		群光電能科技股份有限公司
第25爬完		九暘電子股份有限公司
第26爬完		宏觀眼科診所
第27爬完		中華電信股份有限公司
第28爬完		環球晶圓股份有限公司
第29爬完		中鋼機械股份有限公司
第30爬完		祥碩科技股份有限公司
第31爬完		廣宇科技股份有限公司
第32爬完		致伸科技股份有限公司
第33爬完		神基科技股份有限公司
第34爬完		智原科技股份有限公司
第35爬完		Macroblock_聚積科技股份有限公司
第36爬完		菱光科技股份有限公司
第37爬完		智易科技股份有限公司
第38爬完		創惟科技股份有限公司
第39爬完		敦泰保全股份有限公司
第40找不到	新興
第41爬完		亞泰金屬工業股份有限公司
第42爬完		中聯資源股份有限公司
第43爬完		松翰科技股份有限公司
第44爬完		台灣鋼鐵股份有限公司
第45爬完		EPSON_台灣愛普生科技股份有限公司
第46爬完		晶心科技股份有限公司
第47爬完		緯穎科技服務股份有限公司
第48爬完		威盛電子股份有限公司
第49爬完		台塑科騰化學有限公司
第50爬完		神盾應用股份有限公司
第

In [17]:
driver_path = "\Dropbox\python\selenium\chromedriver\chromedriver.exe"
driver = webdriver.Chrome(driver_path)
driver.get("https://www.104.com.tw/jobs/main/")
driver.maximize_window()

AttributeError: 'str' object has no attribute 'capabilities'

In [ ]:
company = []
job_name = []
location = []
salary = []
requirement = []
education = []
descirption = []
link = []

In [ ]:
for index, company_link in enumerate(company_dict.values()):
    driver.get(company_link)
    try:    #如果找不到每筆100頁選單，則總頁數為1頁
        driver.find_element_by_css_selector('div:nth-child(2) > label > select > option:nth-child(5)').click()  #選每頁100筆
        time.sleep(2)
        total_page = int(driver.find_element_by_css_selector('div.col.main > div.joblist.rounded > div.joblist__footer > div > div:nth-child(2) > label > select').text[-5:-3]) #找出總共幾頁
    except:
        total_page = 1
    for page in range(1, total_page + 1):  #從第1頁爬到最後一頁
        job_link = company_link[:-24] + 'roleJobCat=0_0&area=0&page={}&pageSize=100&order=8&asc=0&jobsource=joblist_r_cust'.format(page)
        driver.get(job_link)
        time.sleep(1.5)

        for i in range(1,104):            
            try:    #如工作數量少於104個，則略過
                job_name_tag = driver.find_element_by_css_selector("div:nth-child({}) > div > div.info.col > div > div.info-job.text-break.mb-2 > a".format(i)).text
                job_name.append(job_name_tag)

                link_tag = driver.find_element_by_css_selector("div:nth-child({}) > div > div.info.col > div > div.info-job.text-break.mb-2 > a".format(i)).get_attribute("href")
                link.append(link_tag)

                location_tag = driver.find_element_by_css_selector('div:nth-child({}) > div > div.info.col > div > div.info-tags.mb-2.font-weight-bold.gray-deep-dark > span:nth-child(1)'.format(i)).text
                location.append(location_tag)

                requirement_tag = driver.find_element_by_css_selector('div:nth-child({}) > div > div.info.col > div > div.info-tags.mb-2.font-weight-bold.gray-deep-dark > span:nth-child(2)'.format(i)).text
                requirement.append(requirement_tag)

                education_tag = driver.find_element_by_css_selector('div:nth-child({}) > div > div.info.col > div > div.info-tags.mb-2.font-weight-bold.gray-deep-dark > span:nth-child(3)'.format(i)).text
                education.append(education_tag)

                descirption_tag = driver.find_element_by_css_selector('div:nth-child({}) > div > div.info.col > div > div.info-description.text-gray-darker.t4.text-break.mb-2.position-relative'.format(i)).text
                descirption.append(descirption_tag)

                salary_tag = driver.find_element_by_css_selector('div:nth-child({}) > div > div.info.col > div > div.info-othertags.mb-2 > span'.format(i)).text
                salary.append(salary_tag)

                company_name_tag = driver.find_element_by_css_selector('div:nth-child(1) > div.breadcrumb.py-3 > ol > li:nth-child(2) > div > span').text
                company.append(company_name_tag)

            except:
                pass
    print("第" + str(index+1) + '公司爬完')

In [ ]:
data = {
    'company':      company,
    'job_name':     job_name,
    'location':     location,
    'salary':       salary,
    'requirement':  requirement,
    'education':    education,
    'descirption':  descirption,
    'link':         link,
}

df = pd.DataFrame(data)
df.drop_duplicates(subset='job_name',inplace=True)   #job_name一樣內容就會一樣，刪除重複的置頂工作
df.to_excel('104.xlsx', sheet_name= '104')
df

In [ ]:
print(len(company))
print(len(job_name))
print(len(location))
print(len(salary))
print(len(requirement))
print(len(education))
print(len(descirption))
print(len(link))

In [ ]:
end = time.time()
print('完成共花費',round((end - start)/60,2),'分')

In [ ]:
winsound.Beep(300,500)
winsound.Beep(360,500)
winsound.Beep(500,500)
winsound.PlaySound('alert', winsound.SND_ASYNC)